In [1]:
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
import random
import math

In [2]:
def load_mnist(path, kind="train"):
    labels_path = os.path.join(path, '%s-labels.idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images.idx3-ubyte' % kind)
 
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        # 'I'表示一个无符号整数，大小为四个字节
        # '>II'表示读取两个无符号整数，即8个字节
        labels = np.fromfile(lbpath, dtype=np.uint8)
 
    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols =  struct.unpack('>IIII', imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
 
    return images, labels

In [3]:
X_train, y_train = load_mnist("mnist_dataset/", kind="train")
X_test, y_test = load_mnist("mnist_dataset/", kind="t10k")

In [4]:
def get_act(x):
    act_vec = []
    for i in x:
        act_vec.append(1/(1+np.exp(-i)))
    act_vec = np.array(act_vec)
    return act_vec

In [5]:
class NN_Mnist(object):
    
    def __init__(self, sizes):
        self.inp_num = sizes[0]    # 输入层节点数
        self.out_num = sizes[2]    # 输出节点数
        self.hid_num = sizes[1]  # 隐层节点数
        self.w1 = 0.2*np.random.random((self.inp_num, self.hid_num))- 0.1
        self.w2 = 0.2*np.random.random((self.hid_num, self.out_num))- 0.1
        self.hid_offset = np.zeros(self.hid_num)
        self.out_offset = np.zeros(self.out_num)
    def train(self,X_train,y_train,hid_lrate,inp_lrate):
        X_train = np.array(X_train,dtype='float')
        #均值归一化 易于收敛
        X_train/=256
        for count in range(len(X_train)):
            t_label = np.zeros(self.out_num)
            t_label[y_train[count]] = 1
            #前向过程
            
            # 隐层值
            self.hid_value = np.dot(X_train[count], self.w1) + self.hid_offset
            # 隐层激活值
            self.hid_act = get_act(self.hid_value)
            # 输出层值
            self.out_value = np.dot(self.hid_act, self.w2) + self.out_offset
            # 输出层激活值
            self.out_act = get_act(self.out_value)

            #后向过程
            # 输出值与真值间的误差
            cost = t_label - self.out_act
            # 输出层delta计算
            out_delta = cost * self.out_act * (1-self.out_act)
            # 隐层delta计算
            hid_delta = self.hid_act * (1-self.hid_act) * np.dot(self.w2, out_delta)
            # 更新隐层到输出层权向量
            for i in range(0, self.out_num):
                self.w2[:,i] += hid_lrate * out_delta[i] * self.hid_act
            
            # 更新输出层到隐层的权向量
            for i in range(0, self.hid_num):
                self.w1[:,i] += inp_lrate * hid_delta[i] * X_train[count]      
            # 输出层偏置更新
            self.out_offset += hid_lrate * out_delta                             
            self.hid_offset += inp_lrate * hid_delta
        print("Training complete.")
    def error(self,X,y):
        X = np.array(X,dtype='float') 
        X/=256
        temp=0
        err_sample = []
        for count in range(len(X)):
            # 隐层值
            self.hid_value = np.dot(X[count], self.w1) + self.hid_offset 
            # 隐层激活值
            self.hid_act = get_act(self.hid_value)
            # 输出层值
            self.out_value = np.dot(self.hid_act, self.w2) + self.out_offset
            # 输出层激活值
            self.out_act = get_act(self.out_value)
            if np.argmax(self.out_act) == y[count]:
                temp+=1
            else:
                err_temp = []
                err_temp.append(y[count])
                err_sample.append(err_temp)
        with open("err_label.txt",'w') as f:
            for item in err_sample:
                f.write(str(item))
                f.write('\n')
        print('Error is: %.2f%%'%((1-float(temp)/len(X))*100))

In [6]:
network = NN_Mnist([784,30,10])

In [18]:
network.train(X_train,y_train,0.2,0.2)
#network.error(X_train,y_train)

Training complete.


In [19]:
network.error(X_test,y_test)

Error is: 6.76%


In [51]:
network.train(X_test,y_test,0.2,0.2)
network.error(X_test,y_test)

Training complete.
Error is: 3.93%


In [20]:
Network = open("MyNetWork_order.txt", 'w')
Network.write(str(network.inp_num))
Network.write('\n')
Network.write(str(network.hid_num))
Network.write('\n')
Network.write(str(network.out_num))
Network.write('\n')
Network.write(str(0.2)) 
Network.write('\n')      
Network.write(str(0.2)) 
Network.write('\n')      
              
for i in network.w1:
    for j in i:
        Network.write(str(j))
        Network.write(' ')
    Network.write('\n')

for i in network.w2:
    for j in i:
        Network.write(str(j))
        Network.write(' ')
Network.write('\n')

for i in network.hid_offset:
    Network.write(str(i))
    Network.write(' ')
Network.write('\n')

for i in network.out_offset:
    Network.write(str(i))
    Network.write(' ')
Network.write('\n')
Network.close()

In [27]:
from PIL import Image
def MatrixToImage(data):
    data = data*255
    new_im = Image.fromarray(data.astype(np.uint8))
    return new_im

In [43]:
test = np.array(X_test[1])
test = test.reshape(28,28)
from PIL import Image
im = Image.fromarray(test)
im.save("b.bmp")

In [53]:
network.error(X_train,y_train)

Error is: 4.29%


In [21]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)